<a href="https://colab.research.google.com/github/nxf-emma/APS360-Artificial-Intelligence-Fundamentals/blob/main/New_Structure_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
import os
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

img_size = 48 

In [ ]:
#Convolutional Neural Network Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.pool = nn.MaxPool2d(2, 2) #kernel_size, stride 

        self.conv1 = nn.Conv2d(3, 5, 3, padding = 2) #in_channels, out_chanels, kernel_size
        self.conv2 = nn.Conv2d(5, 25, 3) #in_channels, out_chanels, kernel_size
        self.conv3 = nn.Conv2d(25, 50, 3)
        self.conv4 = nn.Conv2d(50, 100, 3)
        self.conv5 = nn.Conv2d(100, 125, 3)
        
        self.fc1 = nn.Linear(10*1*1, 32) #chanels*dimension
        self.fc2 = nn.Linear(32, 7) #last output channel always = 7

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # 50x50
        x = self.pool(F.relu(self.conv2(x)))
        # 24x24
        x = self.pool(F.relu(self.conv3(x)))
        # 11x11
        x = self.pool(F.relu(self.conv4(x)))
        # 4x4
        x = self.pool(F.relu(self.conv5(x)))
        # 1x1

        x = x.view(-1, 10*1*1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1)
        return x

In [ ]:
def get_accuracy(model, dataloader):
    

    correct = 0
    total = 0
    for imgs, labels in dataloader:
        
        if use_cuda and torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()

        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [9]:
def train(model, train_data, val_data, batch_size=2000, num_epochs=10, lr = 0.01):
    
    num_workers = 1
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    iters, losses, train_acc, val_acc = [], [], [], []

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        print("Epoch", epoch)
        n = 0
        for imgs, labels in iter(train_loader):
            print("iteration ", n)

            if use_cuda and torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()

            out = model(imgs)             # forward pass

            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
            n += 1

        # save the current training information
    iters.append(epoch)
    losses.append(float(loss)/batch_size)             # compute *average* loss
    # calculate accuracy at each epoch instead of iteration
    # def get_accuracy(model, data):
    train_acc.append(get_accuracy(model, train_loader)) # compute training accuracy 
    # compute validation acc every epoch
    val_acc.append(get_accuracy(model, val_loader))  # compute validation accuracy
    print("Epoch: ", epoch, " Validation Accuracy: ", val_acc[epoch])

            

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

In [ ]:
train_path = '/content/drive/MyDrive/APS360 Team/aps360_dataset/FER-2013_Tensors/train'
val_path = '/content/drive/MyDrive/APS360 Team/aps360_dataset/FER-2013_Tensors/val'
test_path = '/content/drive/MyDrive/APS360 Team/aps360_dataset/FER-2013_Tensors/test'

train_data = datasets.DatasetFolder(train_path, loader=torch.load, extensions=('.tensor'))
val_data = datasets.DatasetFolder(val_path, loader=torch.load, extensions=('.tensor'))
test_data = datasets.DatasetFolder(test_path, loader=torch.load, extensions=('.tensor'))

In [ ]:
#train
model = CNN()
use_cuda = True
model.cuda()
